# Kurs Geoprocessing mit Python und Opensource Komponenten 2023 ETH Zürich

1. bis 3.3. 2023

## TEIL I
1.3.2023

Umprojizierung Gemeinden von LV03 nach WGS84

In [ ]:
import os
os.system('ogr2ogr -f "Esri Shapefile" -t_srs "EPSG:4326" \
          Data/GEM_SO_WGS84.shp Data/Gemeinden_Solothurn.shp -overwrite')

Umwandlung in KML:

In [ ]:
import os
os.system('ogr2ogr -f "KML" Data/GEM_SO.kml Data/Gemeinden_Solothurn.shp')

# Arbeiten mit Vektordaten

In [ ]:
from osgeo import ogr
import os

drv = ogr.GetDriverByName("Esri Shapefile")
path2ds = os.path.join("Data","Gemeinden_Solothurn.shp")
print(path2ds)

datasource = drv.Open(path2ds)
layer = datasource.GetLayer()
#print(layer)

numFtrs = layer.GetFeatureCount()
print(f"Anzahl Features: {numFtrs}")


**so nicht!!**:

fuerSaraina = ogr.GetDriverByName("Esri Shapefile").Open(os.path.join("Data","Gemeinden_Solothurn.shp")).GetLayer(0).GetFeatureCount()
print(fuerSaraina)

In [ ]:
layerDef = layer.GetLayerDefn()
numAtts = layerDef.GetFieldCount()
print(f"Anzahl Attribute: {numAtts}")


In [ ]:
for i in range(numAtts):
    fieldDef = layerDef.GetFieldDefn(i)
    print(f"Attributname: {fieldDef.GetName()}, Typ: {fieldDef.GetType()}")

In [ ]:
spatialRef = layer.GetSpatialRef()
#print(f"Layer '{layer.GetName()}' hat das SRS: \n{spatialRef}")
print("Layer '%s' hat das SRS: \n%s mit %s Features." %(layer.GetName(),spatialRef,numFtrs) )

In [ ]:
#Raeumliche Ausdehnung:
extent = layer.GetExtent()
print(extent)

In [ ]:
ulx = extent[0]
uly = extent[2]
orx = extent[1]
ory = extent[3]
print(f"Ausdehnung des Datensatzes '{layer.GetName()}':\nUnten links: ({ulx},{uly}) und oben rechts: ({orx},{ory})")

In [ ]:
print('Unten-links:', extent[0], extent[2])
print('Oben-rechts:', extent[1], extent[3])

GetGeomType()

In [ ]:
GeomType = ogr.GeometryTypeToName(layer.GetGeomType())
print("Geometrietyp: %s" %GeomType)

Analyse Weltdatensatz **"TM_WORLD_BORDERS-0.3.shp"**

In [ ]:
from osgeo import ogr
import os 

def metadataFromGisLayer(lyr,printOption=False):
    drv = ogr.GetDriverByName("ESRI Shapefile")
    path2ds = lyr
    datasource = drv.Open(path2ds)

    mylayer = datasource.GetLayer(0)
    print(f"Nun gesetzt: {mylayer.GetName()}")

    ftrcnt = mylayer.GetFeatureCount()
    mylayerDef = mylayer.GetLayerDefn()
    numatts = mylayerDef.GetFieldCount()
    if printOption:
        print("Der Datensatz '%s' hat %i Länder mit je %i Attributen" %(mylayer.GetName(),int(ftrcnt),int(numatts)))
    for i in range(numatts):
        fieldDef = mylayerDef.GetFieldDefn(i)
        if printOption:
            print("Attribut  %s ist vom Typ %s" %(fieldDef.GetName(),fieldDef.GetType()))
    sRs = mylayer.GetSpatialRef()
    if printOption:
        print("Layer %s hat folgendes Räumliches Bezugssytem: %s" %(mylayer.GetName(),sRs))

    extent = mylayer.GetExtent()
    if printOption:
        print(extent)
    ulx = round(float(extent[0]),4)
    uly = round(float(extent[2]),4)
    orx = round(float(extent[1]),4)
    ory = round(float(extent[3]),4)
    if printOption:
        print(f"Ausdehnung des Datensatzes '{mylayer.GetName()}':\nUnten links: ({ulx},{uly}) und oben rechts: ({orx},{ory})")

        
        

In [ ]:
myLyr = "Data/TM_WORLD_BORDERS-0.3.shp"
lyrInstance = metadataFromGisLayer(myLyr)

## TEIL II
2.3.2023
<p>Hilfreich: <a href='https://pcjericks.github.io/py-gdalogr-cookbook/layers.html'>https://pcjericks.github.io/py-gdalogr-cookbook/layers.html</a>

In [ ]:
path2ds = os.path.join("Data","Gemeinden_Solothurn.shp")
lyrInstance = metadataFromGisLayer(path2ds)

In [ ]:
from osgeo import ogr, os

drv = ogr.GetDriverByName("ESRI Shapefile")
path2ds = os.path.join("Data","Gemeinden_Solothurn.shp")
datasource = drv.Open(path2ds)
mylayer = datasource.GetLayer(0)

feature = mylayer.GetFeature(12)
attributes = feature.items()
for key,value in attributes.items():
    print(f"{key}: {value}")

Geometrieinformation über: GetGeometryRef()

In [ ]:
myGeometry = feature.GetGeometryRef()
#print("Geometrietyp: %s" %geometry)
print(myGeometry.GetGeometryName())
print(myGeometry.GetPointCount())
print(myGeometry.GetGeometryCount())

In [ ]:
def analyzeGeometry(geometry, indent=0):
    s = []
    s.append("  " * indent)
    s.append(geometry.GetGeometryName())
    if geometry.GetPointCount() > 0:
        s.append(" mit %d Stuetzpunkten" % geometry.GetPointCount())
    if geometry.GetGeometryCount() > 0:
        s.append(" enthaelt:")

    print ("".join(s))

    for i in range(geometry.GetGeometryCount()):
        analyzeGeometry(geometry.GetGeometryRef(i), indent+1)


In [ ]:
analyzeGeometry(myGeometry)

In [ ]:
from osgeo import ogr, os

drv = ogr.GetDriverByName("ESRI Shapefile")
path2ds = "Data/TM_WORLD_BORDERS-0.3.shp"
datasource = drv.Open(path2ds)
mylayer = datasource.GetLayer(0)

feature = mylayer.GetFeature(0)
attributes = feature.items()
for key,value in attributes.items():
    print(f"{key}: {value}")
myGeometry = feature.GetGeometryRef()
analyzeGeometry(myGeometry)

*Putting it all together...*

In [ ]:
def featureInfo(lyr,placeName=None,printOption=False):
    drv = ogr.GetDriverByName("ESRI Shapefile")
    path2ds = lyr
    datasource = drv.Open(path2ds)
    mylayer = datasource.GetLayer(0)
    if printOption:
        print(f"Nun gesetzt: {mylayer.GetName()}")    

    if not placeName is None:
        #Attributfilter    
        mylayer.SetAttributeFilter("NAME = '%s'" %placeName)
    
    feature = mylayer.GetNextFeature()
    while feature:
        attributes = feature.items()
        for key,value in attributes.items():
            if printOption:
                print("  %s = %s" % (key, value))
        if printOption:
            print()

        #Geometrietyp:
        gtype = feature.GetGeometryRef().GetGeometryName()
        if printOption:
            print("Geometrietyp: %s" %gtype)

        #Geometrie:
        geometry = feature.GetGeometryRef()
        if printOption:
            print("")
        #print "Geometrietyp: %s" %geometry
        analyzeGeometry(geometry)
        if printOption:
            print("")
        feature = mylayer.GetNextFeature()    

In [ ]:
path2ds = os.path.join("Data","Gemeinden_Solothurn.shp")
#metadataFromGisLayer(path2ds)
myPlace2Look4 = None
myPlace = input()
if myPlace != '':
    myPlace2Look4 =myPlace
featureInfo(path2ds,myPlace2Look4,True)

In [ ]:
path2ds = "Data/TM_WORLD_BORDERS-0.3.shp"
metadataFromGisLayer(path2ds)
featureInfo(path2ds,None,True)

*Umprojektion*

In [ ]:
from osgeo import ogr 
from osgeo import osr  
source = osr.SpatialReference() 
source.ImportFromEPSG(2927)  

target = osr.SpatialReference() 
target.ImportFromEPSG(4326)  

transform = osr.CoordinateTransformation(source, target)  

origpnt = "POINT (1120351.57 741921.42)"
transfpoint = ogr.CreateGeometryFromWkt(origpnt) 
transfpoint.Transform(transform) 
print("Ausgangskoordinaten: %s" %origpnt)
print("Transformierte Koordinaten: %s" %transfpoint.ExportToWkt())

In [ ]:
source = osr.SpatialReference() 
source.ImportFromEPSG(21781)  
transform = osr.CoordinateTransformation(source, target)  

origpnt = "POINT (600000.00 200000.00)"
transfpoint = ogr.CreateGeometryFromWkt(origpnt) 
transfpoint.Transform(transform) 
print("Ausgangskoordinaten: %s" %origpnt)
print("Transformierte Koordinaten: %s" %transfpoint.ExportToWkt())

In [ ]:
def reProj(pnt,s,d):
    source = osr.SpatialReference() 
    source.ImportFromEPSG(s)  
    target = osr.SpatialReference() 
    target.ImportFromEPSG(d) 
    transform = osr.CoordinateTransformation(source, target)  
    transfpoint = ogr.CreateGeometryFromWkt(pnt) 
    transfpoint.Transform(transform) 
    return transfpoint.ExportToWkt()

origPnt = "POINT (600000.00 200000.00)"  
reProjPnt = reProj(origPnt,21781,4326).split(" ")


reProjPntX = reProjPnt[1].split("(")[1]
reProjPntY = reProjPnt[2].split(")")[0]
print(f"Transformierte Koordinaten: x={reProjPntX}, y={reProjPntY}")

# Umgang mit Rasterdaten

In [ ]:
from osgeo import gdal

fn = 'Data/ortho14_5m_rgb_solothurn.tif'
ds = gdal.Open(fn)
if ds is None:
    print ('Datensatz %s konnte nicht geöffnet werden!' %fn)
    sys.exit(1)

#Dimension des Rasterbildes
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount

print ("Anzahl Spalten: %d" %cols)
print ("Anzahl Zeilen: %d" %rows)
print ("Anzahl Baender: %d" %bands)

In [ ]:
gTr = ds.GetGeoTransform()
print(gTr)

In [ ]:
geotransform = ds.GetGeoTransform()
if not geotransform is None:
    print ('Origin = (',geotransform[0], ',',geotransform[3],')')
    print ('Pixel Size = (',geotransform[1], ',',geotransform[5],')')
    print ('Rotation: ',geotransform[2], ' / ',geotransform[4],'')


In [ ]:
from osgeo import gdal
import sys

def rasterInfo(rD,outFl,printOption=False):
    ds = gdal.Open(rD)
    if ds is None:
        print ("Datensatz '%s' konnte nicht geöffnet werden!" %rD)
        sys.exit()

    cols = ds.RasterXSize
    rows = ds.RasterYSize
    bands = ds.RasterCount
    if printOption:
        outTxt = f"'{rD}' hat {cols} Spalten, {rows} Zeilen \n und Anzahl Bänder: {bands}."
        with open(outFl,'a') as outF:
            outF.write(outTxt)
        print(outTxt)

*Iteriere über alle Dateien von einem bestimmten Typ in einem Verzeichnis (inkl Unterverzeichnisse)*:

In [ ]:
for root, dirs, files in os.walk(r'X:\_GeoProc2023\Data'):
    # select file name
    for file in files:
        # check the extension of files
        if file.endswith('.tif'):
            rasterInfo(os.path.join(root, file),True)

*Ausdehnung Raster*

In [ ]:
geotransform = ds.GetGeoTransform()
if not geotransform is None:
    print ('Origin = (',geotransform[0], ',',geotransform[3],')')
    print ('Pixel Size = (',geotransform[1], ',',geotransform[5],')')
    print ('Rotation: ',geotransform[2], ' / ',geotransform[4],'')


*Abfrage von radiometrischen Werten eines Bildes anhand von Koordinaten*

In [ ]:
# obtained from http://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides4.pdf and adapted
# script to get pixel values at a set of coordinate by reading in one pixel at a time

import os, sys, numpy, time
from osgeo import gdal
from osgeo.gdalconst import *

# start timing
startTime = time.time()
# coordinates to get pixel values for
xValues = [594000.0, 604000.0, 613500.0,599594.0]
yValues = [229500.0, 231000.0, 222800.0,226081.0]

# register all of the drivers
gdal.AllRegister()
# open the image
ds = gdal.Open('Data/ortho14_5m_rgb_solothurn.tif', GA_ReadOnly)
if ds is None:
    print ('Could not open image')
    sys.exit()

# get image size
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
# get georeference info
transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]
print('X | Y | xOffset | yOffset | Wert Band 1 | Wert Band 2 | Wert Band 3 ')

# loop through the coordinates
#for i in range(3):
for i in range(len(xValues)):
    # get x,y
    x = xValues[i]
    y = yValues[i]
    # compute pixel offset
    xOffset = int((x - xOrigin) / pixelWidth)
    yOffset = int((y - yOrigin) / pixelHeight)
    # create a string to print out
    s = str(x) + ' ' + str(y) + ' ' + str(xOffset) + ' ' + str(yOffset) + ' '
    # loop through the bands
    for j in range(bands):
        band = ds.GetRasterBand(j+1) # 1-based index
        #read data and add the value to the string
        data = band.ReadAsArray(xOffset, yOffset, 1, 1)
        value = data[0,0]
        #print(data)
        #value2 = numpy.median(data)
        s = s + str(value) + ' '

    #print out the data string
    print(s)
# figure out how long the script took to run
endTime = time.time()
print()
print ('The script took %.3f seconds' %(endTime - startTime))

In [ ]:
# obtained from http://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides4.pdf and adapted
# script to get pixel values at a set of coordinate by reading in one pixel at a time

import os, sys, numpy, time
from osgeo import gdal
from osgeo.gdalconst import *

# start timing
startTime = time.time()
# coordinates to get pixel values for
xValues = [10.1]
yValues = [47.3]

# register all of the drivers
gdal.AllRegister()
# open the image
ds = gdal.Open('Data/Elevation_raster.tif', GA_ReadOnly)
if ds is None:
    print ('Could not open image')
    sys.exit()

# get image size
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
# get georeference info
transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]
print('X | Y | xOffset | yOffset | Höhenwert')

# loop through the coordinates
#for i in range(3):
for i in range(len(xValues)):
    # get x,y
    x = xValues[i]
    y = yValues[i]
    # compute pixel offset
    xOffset = int((x - xOrigin) / pixelWidth)
    yOffset = int((y - yOrigin) / pixelHeight)
    # create a string to print out
    s = str(x) + ' ' + str(y) + ' ' + str(xOffset) + ' ' + str(yOffset) + ' '
    # loop through the bands
    for j in range(bands):
        band = ds.GetRasterBand(j+1) # 1-based index
        #read data and add the value to the string
        data = band.ReadAsArray(xOffset, yOffset, 1, 1)
        value = data[0,0]
        #print(data)
        #value2 = numpy.median(data)
        s = s + str(value) + ' '

    #print out the data string
    print(s)
# figure out how long the script took to run
endTime = time.time()
print()
print ('The script took %.3f seconds' %(endTime - startTime))

In [ ]:
#obtained from http://www.gis.usu.edu/~chrisg/python/2009/lectures/ospy_slides4.pdf and adapted

# script to get pixel values at a set of coordinate
# by reading in one pixel at a time
# modified to read altitude, aspect and sloap from the band

import os, sys, numpy, time
from osgeo import gdal
from osgeo.gdalconst import *
import random

random.seed()

'''
Infos zur Rasterdatei:
Lower Left  (   9.9998611,  47.2498611) (  9d59'59.50"E, 47d14'59.50"N)
Upper Right (  10.2501389,  47.5001389) ( 10d15' 0.50"E, 47d30' 0.50"N)
Pixel Size = (0.000277777777778,-0.000277777777778)
Size is 901, 901
'''
xValues = []
yValues = []
pntnumber = input("Anzahl zuf. Punkte:")
for i in range(1, int(pntnumber)+1):
    #Erzeuge zufällige Koordinatenwerte
    rechts = random.random() * 0.000277777777778 * 901 + 9.9998611
    hoch = random.random() * 0.000277777777778 * 901 + 47.2498611

    # coordinates to get pixel values for
    xValues.append(rechts)
    yValues.append(hoch)

    #print("%i. Punkt: x= %f | y= %f" %(i,rechts, hoch))

# start timing
startTime = time.time()

# register all of the drivers
gdal.AllRegister()

imagefilename = "Data/Elevation_raster.tif"
path = "Data/"

# open the image
ds = gdal.Open(imagefilename) #The band is altitude values

if ds is None:
    print ('Could not open image')
    sys.exit(1)

# Translate to slope and aspect /          #gdaldem hillshade input_dem output_hillshade

# File to store results and open in QGIS
rndpntfl = open("_randompointfilewithDEMinfo.csv","w")

sloapcommand = 'gdaldem slope %s %sEle_slope.tif -s 10000' %(imagefilename,path)
#print ("command to run: %s" %sloapcommand)   #gdaldem slope input_dem output_slope_map

os.system(sloapcommand)

aspectcommand = 'gdaldem aspect %s %sEle_aspect.tif' %(imagefilename,path)
#print ("command to run: %s" %aspectcommand)  #gdaldem aspect input_dem output_aspect_map

os.system(aspectcommand)

da= gdal.Open(path + 'Ele_slope.tif')
dl= gdal.Open(path + 'Ele_aspect.tif')

# get image size
rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount

bandsaspect = da.RasterCount
bandsslope = dl.RasterCount

# get georeference info
transform = ds.GetGeoTransform()
xOrigin = transform[0]
yOrigin = transform[3]
pixelWidth = transform[1]
pixelHeight = transform[5]


print ('Ursprung der Rasterdatei: ', xOrigin , yOrigin)
print("")
print ('X | Y | xOffset | yOffset | Altitude  | Aspect  | Slope |' )
rndpntfl.write('X, Y, xOffset, yOffset, Altitude, Aspect, Slope \n')

# loop through the coordinates
for k in range(len(xValues)):
    x = xValues[k]
    y = yValues[k]

    # compute pixel offset
    xOffset = int((x - xOrigin) / pixelWidth)
    yOffset = int((y - yOrigin) / pixelHeight)

    # create a string to print out
    s = str(x) + ' ' + str(y) + ' ' + str(xOffset) + ' ' + str(yOffset) + ' '

    # loop through the bands

    for j in range(bands):
        band = ds.GetRasterBand(j+1) # 1-based index
        # read data and add the value to the string
        data = band.ReadAsArray(xOffset, yOffset, 1, 1)
        value = data[0,0]
        s = s + str(value) + ' '

    #get aspect value
    for f in range(bandsaspect):
        band_da = da.GetRasterBand(f+1) # 1-based index
        # read data and add the value to the string
        data_da = band_da.ReadAsArray(xOffset, yOffset, 1, 1)
        value_da = data_da[0,0]
        s = s + str(value_da) + ' '

    #get slope value
    for e in range(bandsslope):
        band_dl = dl.GetRasterBand(e+1) # 1-based index

        # read data and add the value to the string
        data_dl = band_dl.ReadAsArray(xOffset, yOffset, 1, 1)
        value_dl = data_dl[0,0]

        s = s + str(value_dl) + ' \n'

    #print out the data string
    s2=s.replace("\n","")
    print (s2)
    rndpntfl.write(s.replace(" ",", "))
    #print ("")


# figure out how long the script took to run
endTime = time.time()
print ('The script took ' + str(endTime - startTime) + ' seconds')
rndpntfl.close()

In [ ]:
https://github.com/myproxxy/ETHGeodataProcessingOSSPublic

## TEIL III
3.3.2023

DEM-relevante Produkte erstellen: Höhenlinien, Ausrichtung, Hangneigung, Schummerung

In [ ]:
import os

path =r'x:\_GeoProc2023\Data'
eleR = os.path.join(path,'Elevation_raster.tif')
eleC = os.path.join(path,'contour_1000.shp')
cmd = f'gdal_contour –a alti –i 1000.0 {eleR} {eleC}'
print(cmd)
os.system(cmd)

*Abgeleitete Produkte des Höhenmodells*

In [ ]:
import os

imagefilename = "Data/Elevation_raster.tif"
path = "Data/"

#Neigung
sloapcommand = 'gdaldem slope %s %sEle_slope.tif -s 10000' %(imagefilename,path)
print ("command to run: %s" %sloapcommand)   #gdaldem slope input_dem output_slope_map
os.system(sloapcommand)

#Ausrichtung
aspectcommand = 'gdaldem aspect %s %sEle_aspect.tif' %(imagefilename,path) 
print ("command to run: %s" %aspectcommand)  #gdaldem aspect input_dem output_aspect_map
os.system(aspectcommand) 

#Schummerung
hillshadecommand = 'gdaldem hillshade %s %sEle_hillshade.tif' %(imagefilename,path) 
print ("command to run: %s" %hillshadecommand)  #gdaldem aspect input_dem output_aspect_map
os.system(hillshadecommand) 

*Extraktion eines Teilbereichs aus einer Rasterdatei*

In [ ]:
import os, sys
from osgeo import gdal
from osgeo.gdalconst import *

# register all of the drivers
gdal.AllRegister()

#Open Rasterfile
fn = 'Data/worldmap.jpg'
ds = gdal.Open(fn)
if ds is None:
    print ('Datensatz %s konnte nicht geöffnet werden' %fn)
    sys.exit(1)

os.system('gdalinfo %s' %fn) 
translatecommand = 'gdal_translate -projwin 1680 170 2200 550 %s Data/europe.tif' %fn
print ("command to run: %s" %translatecommand )
os.system(translatecommand)

#kleinere Kopie von Europa
translatecommand = 'gdal_translate -projwin 1680 170 2200 550 -outsize 50%% 50%% %s Data/europesmall.tif' %fn
print ("command to run: %s" %translatecommand) 
os.system(translatecommand)

print ("")
os.system('gdalinfo Data/europe.tif')
print ("")
os.system('gdalinfo Data/europesmall.tif')

In [ ]:
import os, sys
from osgeo import gdal
from osgeo.gdalconst import *

# register all of the drivers
gdal.AllRegister()

#Open Rasterfile
fn = 'Data/ortho14_5m_rgb_solothurn.tif'
ds = gdal.Open(fn)
if ds is None:
    print ('Datensatz %s konnte nicht geöffnet werden' %fn)
    sys.exit(1)

os.system('gdalinfo %s' %fn) 
translatecommand = 'gdal_translate -of GTiff -a_srs EPSG:21781 -projwin 602599 228251 605552 226823 %s Data/soloAus.tif' %fn
print ("command to run: %s" %translatecommand )
os.system(translatecommand)


translatecommand = 'gdal_translate -of GTiff -srcwin 2500 1600 1000 1000 %s Data/soloAus1000.tif' %fn
print ("command to run: %s" %translatecommand )
os.system(translatecommand)


## Geodaten schreiben/erzeugen

Projektionssystem festlegen

In [ ]:
from osgeo import osr
srs = osr.SpatialReference()
#srs.SetWellKnownGeogCS('WGS84')
srs.SetFromUserInput("EPSG:21781")

prj1=srs.ExportToWkt()
print(prj1)
print()
prj2=srs.ExportToPrettyWkt()
print(prj2)
print()
prj3=srs.ExportToProj4()
print(prj3)

In [ ]:
from osgeo import ogr

driver = ogr.GetDriverByName("Esri Shapefile")    #gemäss http://gdal.org/1.11/ogr/ogr_formats.html

if os.path.exists("Data/_Quadrat.shp"):
    driver.DeleteDataSource("Data/_Quadrat.shp") 
destinationFile = driver.CreateDataSource("Data/_Quadrat.shp")
destinationLayer = destinationFile.CreateLayer("Layer", srs)


In [ ]:
fieldDef = ogr.FieldDefn("Name",ogr.OFTString)
fieldDef.SetWidth(80)
destinationLayer.CreateField(fieldDef)

fieldDef = ogr.FieldDefn("Bemerkung", ogr.OFTString)
fieldDef.SetWidth(100)
destinationLayer.CreateField(fieldDef)

fieldDef = ogr.FieldDefn("ID",ogr.OFTInteger)
destinationLayer.CreateField(fieldDef)

In [ ]:
ftrName = "Erstes Feature"
minEasting = 10
maxEasting = 20
minNorthing = 15
maxNorthing = 25

#Definition des OGR Geometrieobjekts als LinearRing
linearRing = ogr.Geometry(ogr.wkbLinearRing)


In [ ]:
#Hinzufügen der Stützpunkte des LinearRing
linearRing.AddPoint(minEasting, minNorthing)
linearRing.AddPoint(maxEasting, minNorthing)
linearRing.AddPoint(maxEasting, maxNorthing)
linearRing.AddPoint(minEasting, maxNorthing)
linearRing.AddPoint(minEasting, minNorthing)

In [ ]:
#Instanzieren der Geometrie als WKBPolygon ins sqr Objekt
sqr = ogr.Geometry(ogr.wkbPolygon)
sqr.AddGeometry(linearRing)

In [ ]:
sqrfeature = ogr.Feature(destinationLayer.GetLayerDefn())
sqrfeature.SetGeometry(sqr)

In [ ]:
#Neues Feature erhält für das Attribut Name den Wert "Erstes Feature" 
sqrfeature.SetField("Name", ftrName)
sqrfeature.SetField("Bemerkung", 'keine Besonderheit')
sqrfeature.SetField("Id", 1)

In [ ]:
destinationLayer.CreateFeature(sqrfeature)
sqrfeature.Destroy()

In [ ]:
print("Erstellung abgeschlossen")
destinationFile.Destroy()

In [ ]:
from osgeo import ogr,osr
import os

#Definition SRS
srs = osr.SpatialReference()
srs.SetWellKnownGeogCS('WGS84')

#Erstellen der neuen Ebene/Layer
#driver = ogr.GetDriverByName("Esri Shapefile2")
driver = ogr.GetDriverByName("GeoJSON")
newLyr = "Data/Donut.geojson"
if os.path.exists(newLyr):
    driver.DeleteDataSource(newLyr) 
#destinationFile = driver.CreateDataSource("Daten/NeuerLayer.shp")
destinationFile = driver.CreateDataSource(newLyr)
destinationLayer = destinationFile.CreateLayer("Layer", srs)

#Festlegung der Attribute
fieldDef = ogr.FieldDefn('Id', ogr.OFTInteger)
destinationLayer.CreateField(fieldDef)
fieldDef = ogr.FieldDefn("Name",ogr.OFTString)
fieldDef.SetWidth(80)
destinationLayer.CreateField(fieldDef)
fieldDef = ogr.FieldDefn("Bemerkung", ogr.OFTString)
fieldDef.SetWidth(100)
destinationLayer.CreateField(fieldDef)

#Erstellen eines Features
ftrName = "Erstes Feature"
ftrBem = "Dies ist mein erstes selbst erstelltes Features"

#geometry = feature.GetGeometryRef()
minEasting = 10
maxEasting = 20
minNorthing = 15
maxNorthing = 25

#Definition des OGR Geometrieobjekts als LinearRing
linearRing = ogr.Geometry(ogr.wkbLinearRing)
#Hinzufügen der Stützpunkte des LinearRing
linearRing.AddPoint(minEasting, minNorthing)
linearRing.AddPoint(maxEasting, minNorthing)
linearRing.AddPoint(maxEasting, maxNorthing)
linearRing.AddPoint(minEasting, maxNorthing)
linearRing.AddPoint(minEasting, minNorthing)

#Definition des OGR Geometrieobjekts als LinearRing
linearRing2 = ogr.Geometry(ogr.wkbLinearRing)
#Hinzufügen der Stützpunkte des LinearRing
linearRing2.AddPoint(minEasting+3, minNorthing+3)
linearRing2.AddPoint(maxEasting-3, minNorthing+3)
linearRing2.AddPoint(maxEasting-3, maxNorthing-3)
linearRing2.AddPoint(minEasting+3, maxNorthing-3)
linearRing2.AddPoint(minEasting+3, minNorthing+3)

#Definition des OGR Geometrieobjekts als LinearRing
linearRing3 = ogr.Geometry(ogr.wkbLinearRing)
#Hinzufügen der Stützpunkte des LinearRing
linearRing3.AddPoint(minEasting+30, minNorthing+30)
linearRing3.AddPoint(maxEasting+30, minNorthing+30)
linearRing3.AddPoint(maxEasting+30, maxNorthing+30)
linearRing3.AddPoint(minEasting+30, maxNorthing+30)
linearRing3.AddPoint(minEasting+30, minNorthing+30)

#Definition des OGR Geometrieobjekts als LinearRing
linearRing4 = ogr.Geometry(ogr.wkbLinearRing)
#Hinzufügen der Stützpunkte des LinearRing
linearRing4.AddPoint(minEasting+8, minNorthing+8)
linearRing4.AddPoint(maxEasting+25, minNorthing+8)
linearRing4.AddPoint(maxEasting+25, maxNorthing+25)
linearRing4.AddPoint(minEasting+8, maxNorthing+25)
linearRing4.AddPoint(minEasting+8, minNorthing+8)


#Definition des OGR Geometrieobjekts als LinearRing
linearRing5 = ogr.Geometry(ogr.wkbLinearRing)
#Hinzufügen der Stützpunkte des LinearRing
linearRing5.AddPoint(minEasting+3, minNorthing+3)
linearRing5.AddPoint(maxEasting-3, minNorthing+3)
linearRing5.AddPoint(maxEasting-3, maxNorthing-3)
linearRing5.AddPoint(minEasting+3, maxNorthing-3)
linearRing5.AddPoint(minEasting+3, minNorthing+3)

#Instanzieren der Geometrie als WKBPolygon ins sqr Objekt
sqr = ogr.Geometry(ogr.wkbPolygon)
#Zuweisen der Geometrie zum instanzierten Objekt
sqr.AddGeometry(linearRing)
sqr.AddGeometry(linearRing2)
sqr.AddGeometry(linearRing3)
sqr.AddGeometry(linearRing4)
#sqr.AddGeometry(linearRing5)
#Neues Feature erhält Attributdefinition
sqrfeature = ogr.Feature(destinationLayer.GetLayerDefn())
#Neues Feature erhält Geometrie
sqrfeature.SetGeometry(sqr)
#Neues Feature erhält für das Attribut Name den Wert "Erstes Feature" 
sqrfeature.SetField("Id", 1)
sqrfeature.SetField("Name", ftrName)
sqrfeature.SetField("Bemerkung", ftrBem)
#Erstellung des Features im neuen Layer
destinationLayer.CreateFeature(sqrfeature)
sqrfeature.Destroy()

print("Erstellung abgeschlossen")
destinationFile.Destroy()

## Verbindung zu einer räumlichen Datenbank

In [133]:
#import gdal
import psycopg2

#Connect to PostgreSQL 
database = "postgis_db"
hst = "ikgpgis.ethz.ch"
usr = "casuser"
pwd = "mrPope-2323-IKG"
prt = "5432"

connection = psycopg2.connect(dbname=database,host=hst,user=usr,password=pwd,port=prt)

#Query Database 
cursor = connection.cursor()
   
sqlStmt = "select name, gmde_nr from gemeinden_solothurn where name like 'O%' order by gmde_nr desc;"
cursor.execute(sqlStmt)
for name, gmdNr in cursor:
    print(f"Gemeinde {name} hat die Nummer {gmdNr}")
    
connection.commit()
    

Gemeinde Olten hat die Nummer 2581
Gemeinde Oberdorf hat die Nummer 2553
Gemeinde Oekingen hat die Nummer 2529
Gemeinde Obergerlafingen hat die Nummer 2528
Gemeinde Obergösgen hat die Nummer 2497
Gemeinde Oensingen hat die Nummer 2407
Gemeinde Oberbuchsiten hat die Nummer 2406


In [139]:
sqlStmt = "DROP TABLE IF EXISTS ogemeinden;"
cursor.execute(sqlStmt)
connection.commit()

sqlStmt = "create table ogemeinden as select * from gemeinden_solothurn where name like 'O%' order by gmde_nr desc;"
cursor.execute(sqlStmt)
connection.commit()

sqlStmt = "select name, gmde_nr from ogemeinden;"
cursor.execute(sqlStmt)
for name, gmdNr in cursor:
    print(f"Gemeinde {name} hat die Nummer {gmdNr}")


Gemeinde Olten hat die Nummer 2581
Gemeinde Oberdorf hat die Nummer 2553
Gemeinde Oekingen hat die Nummer 2529
Gemeinde Obergerlafingen hat die Nummer 2528
Gemeinde Obergösgen hat die Nummer 2497
Gemeinde Oensingen hat die Nummer 2407
Gemeinde Oberbuchsiten hat die Nummer 2406


In [ ]:
sqlstmt="select gmde_nr,name,st_area(geom) as flaeche from gemeinden_solothurn limit 5;"
cursor.execute(sqlstmt)

for tick,trick,track in cursor:
    print(f"Gemeinde {track} {trick} {tick}")


In [ ]:
sqlstmt="select gmde_nr,name,st_area(geom) as flaeche, st_astext(geom) as geometrie \
    from gemeinden_solothurn order by 3 asc limit 1;"
cursor.execute(sqlstmt)

for gemnr,name,flaeche, geom in cursor:
    print(f"Gemeinde {name} {gemnr} {flaeche} {geom}")


## Webdienste ansprechen

In [140]:
# -*- coding: utf-8 -*-

import os, shutil, sys
import urllib.request
from osgeo import gdal
from osgeo import gdalconst


def download(url, dest, fileName=None):
#based on:
#http://stackoverflow.com/questions/862173/how-to-download-a-file-using-python-in-a-smarter-way/863017#863017

    print("******")
    print(url)
    print("******")
    r= urllib.request.urlopen(url)

    fileName = os.path.join(dest, fileName)
    with open(fileName, 'wb') as f:
        shutil.copyfileobj(r,f)
    r.close()

if __name__=='__main__':

    wmsfile = "sogis.gif"
    if os.path.exists(wmsfile):
        os.remove(wmsfile)

    path2save2 = "Data/" #Zielpfad
    wmslink = "https://geo.so.ch/api/wms?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&FORMAT=image%2Fpng&TRANSPARENT=true&LAYERS=ch.so.agi.hintergrundkarte_farbig&STYLES=&SRS=EPSG%3A2056&CRS=EPSG%3A2056&TILED=false&DPI=96&OPACITIES=255&t=675&WIDTH=1920&HEIGHT=710&BBOX=2607051.2375,1228517.0374999999,2608067.2375,1228892.7458333333"
    
    download(wmslink,path2save2,wmsfile)

    #Open Rasterfile
    fn = os.path.join(path2save2,wmsfile)
    ds = gdal.Open(fn, GA_ReadOnly)
    if ds is None:
        print('Could not open ' + fn)
        sys.exit(1)

    #check with gdalinfo
    os.system('gdalinfo %s' %fn)


******
https://geo.so.ch/api/wms?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetMap&FORMAT=image%2Fpng&TRANSPARENT=true&LAYERS=ch.so.agi.hintergrundkarte_farbig&STYLES=&SRS=EPSG%3A2056&CRS=EPSG%3A2056&TILED=false&DPI=96&OPACITIES=255&t=675&WIDTH=1920&HEIGHT=710&BBOX=2607051.2375,1228517.0374999999,2608067.2375,1228892.7458333333
******


*WFS*

In [143]:
# -*- coding: utf-8 -*-
import os, sys, shutil
import urllib.request
from osgeo import ogr
from osgeo import gdal
from osgeo import gdalconst

#WFS Daten von:
#http://www.are.zh.ch/internet/baudirektion/are/de/geoinformation/geodienste_uebersicht/WebFeatureService.html

def download(url, dest, fileName=None):
#based on: 
#http://stackoverflow.com/questions/862173/how-to-download-a-file-using-python-in-a-smarter-way/863017#863017
    print("******")
    print ("Start downloading of %s" %url)
    print("******")

    r= urllib.request.urlopen(url)

    try:
        fileName = os.path.join(dest, fileName)
        with open(fileName, 'wb') as f:
            shutil.copyfileobj(r,f)
        print ("Saved in %s" %fileName)
    finally:
        r.close()
        
def convert2shp(path2save2,wfsfile,outputshapefile):
    fn = os.path.join(path2save2,wfsfile)
    
    driver = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(outputshapefile):
        driver.DeleteDataSource(outputshapefile) 
    
    #convert GMLfile to shape - if needed...
    ogr2ogrstring = 'ogr2ogr -f "ESRI Shapefile" %s %s' %(outputshapefile,fn)
    print (ogr2ogrstring)
    os.system(ogr2ogrstring)
    print ("Conversion successful...")
    
    #... oder GML... 
    wfsfile = ogr.Open(fn)
    if wfsfile is None:
        print ("Datensatz konnte nicht geoeffnet werden.\n")
        sys.exit( 1 )
    
    layer = wfsfile.GetLayer(0)
    lname = layer.GetName()
    
    print ("Layername: ", lname)
    
    #Print out number of records:
    numftrs = layer.GetFeatureCount()
    print ("Anzahl Features in GML Datei: %d" %numftrs)
    print ("")
    
    print ("Count Field Count", layer.GetLayerDefn().GetFieldCount())	
    for feat in range(numftrs):
        for i in range(layer.GetLayerDefn().GetFieldCount()):
            field_defn = layer.GetLayerDefn().GetFieldDefn(i)
        try:
            print ("  %s: %s" %(field_defn.GetName(), layer.GetFeature(feat).GetField(i)))
        except:
            pass
        print() 
    
    
    #Get Extent
    extent = layer.GetExtent()
    print ("Ausdehnung:", extent)
    print ("Oben-links:", extent[0], extent[3])
    print ("Unten-rechts:", extent[1], extent[2])


if __name__=='__main__':

    ######################   
    #Punktdaten:     
    wfsfile = "testpoints.gml"
    outputshapefile = 'wfstestpoints.shp'
    path2save2 = "Data/"
    wfsurl = "http://maps.zh.ch/wfs/HaltestellenZHWFS?SERVICE=WFS&VERSION=1.1.0&Request=getfeature&TYPENAME=haltestellen&MAXFEATURES=30"
    download(wfsurl,path2save2 ,wfsfile)
    
    convert2shp(path2save2,wfsfile,os.path.join(path2save2,outputshapefile))

    #Liniendaten:     
    wfsfile = "testlines.gml"
    outputshapefile = 'wfstestlines.shp'
    path2save2 = "Data/"
    wfsurl = "http://maps.zh.ch/wfs/GemZHWFS?SERVICE=WFS&VERSION=1.1.0&Request=getfeature&TYPENAME=grenzen&MAXFEATURES=100"

    download(wfsurl,path2save2 ,wfsfile)
    convert2shp(path2save2,wfsfile,os.path.join(path2save2,outputshapefile))

******
Start downloading of http://maps.zh.ch/wfs/HaltestellenZHWFS?SERVICE=WFS&VERSION=1.1.0&Request=getfeature&TYPENAME=haltestellen&MAXFEATURES=30
******
Saved in Data/testpoints.gml
ogr2ogr -f "ESRI Shapefile" Data/wfstestpoints.shp Data/testpoints.gml
Conversion successful...
Layername:  haltestellen
Anzahl Features in GML Datei: 30

Count Field Count 17






























Ausdehnung: (2678007.0, 2686463.0, 1245094.0, 1253390.0)
Oben-links: 2678007.0 1253390.0
Unten-rechts: 2686463.0 1245094.0
******
Start downloading of http://maps.zh.ch/wfs/GemZHWFS?SERVICE=WFS&VERSION=1.1.0&Request=getfeature&TYPENAME=grenzen&MAXFEATURES=100
******
Saved in Data/testlines.gml
ogr2ogr -f "ESRI Shapefile" wfstestlines.shp Data/testlines.gml
Conversion successful...
Layername:  grenzen
Anzahl Features in GML Datei: 100

Count Field Count 5




































































































Ausdehnung: (2669244.906, 2716900.4, 1223895.6281